# DB Initialisation Script

This notebook can be used for setting up local Postgres database called 'dsa3101' and for querying the database

To get started, please change the env variables in the next code chunk and simply 'Run All' to load the database.

Ensure that your .venv is activated in this Jupyter environment.

In [11]:
# TOCHANGE

YOUR_POSTGRES_PASSWORD = 'weiyou2000' 
YOUR_POSTGRES_PORT_NO = 5432 

# Default if you have cloned from repo-main 
path_to_products_csv = 'products.csv'
path_to_ratings_csv = 'ratings.csv'
path_to_dailysales_csv = 'daily_sales.csv'
path_to_users_csv = 'users.csv'
path_to_user_behaviour_csv = 'user_behaviour.csv'
host = 'localhost'
database = 'dsa3101' # default db name
user = 'postgres'

## Set Up

In [12]:
#!pip3 install ipython-sql
#!pip3 install sqlalchemy
#!pip3 install psycopg2-binary
#!pip3 install pandas

In [13]:
from sqlalchemy import create_engine, Table, Column, MetaData, String, Numeric, Float, Integer, ForeignKey, Date, ForeignKeyConstraint,DateTime
import pandas as pd
import psycopg2

In [16]:
# Step 1: Connect to the default database (postgres) to check/create the 'dsa3101' database
try:
    # Connect to the 'postgres' database (default)
    conn = psycopg2.connect(
        host=host,
        user=user,
        password=YOUR_POSTGRES_PASSWORD,
        port=YOUR_POSTGRES_PORT_NO,
        database='postgres'  # Connect to 'postgres' instead of 'dsa3101' initially
    )
    conn.autocommit = True  # Enable autocommit so that CREATE DATABASE works
    cursor = conn.cursor()

    # Check if the 'dsa3101' database exists
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{database}';")
    exists = cursor.fetchone()

    # If the database doesn't exist, create it
    if not exists:
        cursor.execute(f'CREATE DATABASE {database};')
        print(f"Database '{database}' has been created.")
    else:
        print(f"Database '{database}' already exists.")
    
    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error while creating or checking the database: {e}")

# Step 2: Now connect to the newly created or existing 'dsa3101' database
try:
    engine = create_engine(f'postgresql://{user}:{YOUR_POSTGRES_PASSWORD}@{host}:{YOUR_POSTGRES_PORT_NO}/{database}')

    # Define MetaData for the DB
    metadata = MetaData()

    # Reflect existing tables from the 'dsa3101' database
    metadata.reflect(bind=engine)
    
    # Drop all tables in the 'dsa3101' database
    metadata.drop_all(bind=engine)
    print("All pre-existing tables have been dropped.")
    
except Exception as e:
    print(f"Error while reflecting or dropping tables: {e}")

finally:
    metadata = MetaData()


Database 'dsa3101' has been created.
All pre-existing tables have been dropped.


## Loading tables into database

In [15]:
# Create and Load  table - products


products_table = Table('products', metadata,
    Column('product_id', String(50), primary_key=True),
    Column('product_name', String(500)),
    Column('about_product', String),  # Use String for TEXT in SQLAlchemy
    Column('category', String(255)),
    Column('actual_price', Numeric(10, 2)),
    Column('discounted_price', Numeric(10, 2)),
    Column('discount_percentage', Numeric(5, 2))
)

# Create the table in the database if it doesn't exist
metadata.create_all(engine)

# Read the CSV file into a DataFrame
products_df = pd.read_csv(path_to_products_csv)

# Insert data into the PostgreSQL table
products_df.to_sql('products', engine, if_exists='append', index=False)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "dsa3101" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [6]:
# Create and Load table - ratings

# Define the 'ratings' table schema
ratings_table = Table('ratings', metadata,
    Column('product_id', String(50), ForeignKey('products.product_id', ondelete='CASCADE'), primary_key=True),
    Column('average_rating', Float),
    Column('review_title', String(1000)),
    Column('review_content', String),  # Use String for TEXT in SQLAlchemy
    Column('rating_count', Integer)
)

# Create the table in the database if it doesn't exist
metadata.create_all(engine)

# Read the CSV file into a DataFrame
ratings_df = pd.read_csv(path_to_ratings_csv)

# Insert data into the PostgreSQL table
ratings_df.to_sql('ratings', engine, if_exists='append', index=False)

351

In [7]:
# Define the 'daily_sales' table schema
daily_sales_table = Table('daily_sales', metadata,
    Column('transaction_id', Integer, primary_key=True),
    Column('date', Date),
    Column('product_id', String(50), ForeignKey('products.product_id', ondelete='CASCADE'), nullable=False),
    Column('quantity', Integer)
)

# Create the table in the database if it doesn't exist
metadata.create_all(engine)

# Read the CSV file into a DataFrame
daily_sales_df = pd.read_csv(path_to_dailysales_csv)

# Insert data into the PostgreSQL table
daily_sales_df.to_sql('daily_sales', engine, if_exists='append', index=False)

887

In [8]:
# Define the 'users' table schema
users_table = Table('users', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('age', Integer),  
    Column('gender', String(50))
)

# Define the 'user_behaviour' table schema
user_behaviour_table = Table('user_behaviour', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('timestamp', DateTime),
    Column('purchase_Frequency', String(50)),
    Column('purchase_Categories', String(255)),
    Column('personalized_Recommendation_Success', String(50)),
    Column('browsing_Frequency', String(50)),
    Column('product_Search_Method', String(50)),
    Column('search_Result_Exploration', String(50)),
    Column('customer_Reviews_Importance', String(1)),  
    Column('add_to_Cart_Browsing', String(50)),
    Column('cart_Completion_Frequency', String(50)),
    Column('cart_Abandonment_Factors', String(255)),
    Column('saveforlater_Frequency', String(50)),
    Column('review_Left', String(50)),
    Column('review_Reliability', String(50)),
    Column('review_Helpfulness', String(50)),
    Column('recommendation_Helpfulness', String(50)),
    Column('personalized_Recommendation_Frequency', String(1)),
    Column('rating_Accuracy', String(1)),  
    Column('shopping_Satisfaction', String(1)),  
    Column('service_Appreciation', String(255)),
    Column('improvement_Areas', String(255)),
    ForeignKeyConstraint(['user_id'], ['users.user_id'], ondelete='CASCADE')
)

# Create the tables in the database if they don't exist
metadata.create_all(engine)

# Read the CSV files into DataFrames
users_df = pd.read_csv(path_to_users_csv)
user_behaviour_df = pd.read_csv(path_to_user_behaviour_csv)

# Insert data into the PostgreSQL tables
users_df.to_sql('users', engine, if_exists='append', index=False)
user_behaviour_df.to_sql('user_behaviour', engine, if_exists='append', index=False)

602